### Initial Setup
Make sure you have the following packages installed:
- pandas
- urllib3
- requests
- datetime

`pip install pandas urllib3 requests datetime`

In [19]:
# pip install pandas urllib3 requests datetime

In [1]:
import pandas as pd
import os
from datetime import date, timedelta

# import our functions
import api_defillama as dfl 
import api_growthe as gtp 
import api_l2beat as l2b

In [2]:
# set filter variables - to help cut down on rows for this demo
trailing_days = 365*5
get_start_of_month = True # True if we want to start on the first day of the month

In [3]:
# compute date range
current_date = date.today()
start_date = current_date - timedelta(days = trailing_days)
if get_start_of_month:
    start_date = start_date.replace(day = 1)
#Format
current_date = pd.to_datetime(current_date)
start_date = pd.to_datetime(start_date)

print(f"Today's Date: {current_date}, Filter Start Date: {start_date}")

Today's Date: 2024-11-04 00:00:00, Filter Start Date: 2019-11-01 00:00:00


### Run Growthepie
[Growthepie](https://www.growthepie.xyz/) provides detailed onchain data (i.e. transactions, revenue, costs, profit) for a set of L2s.

See [API Docs](https://docs.growthepie.xyz/api)

In [4]:
print('getting growthepie metadata')
gtp_md = gtp.get_growthepie_metadata()

print('getting growthepie fundamentals data')
gtp_fun = gtp.get_growthepie_fundamentals()
#filter dates
gtp_fun = gtp_fun[gtp_fun['date']>=start_date]

print('yay')

getting growthepie metadata
getting growthepie fundamentals data
https://api.growthepie.xyz/v1/fundamentals_full.json
yay


In [5]:
num_chains = len(gtp_md[gtp_md['chain_name']!= '-'])
print(f"Growthepie: Total Amount of Chains: {num_chains}")

Growthepie: Total Amount of Chains: 24


In [6]:
gtp_md.head(5)

,name,url_key,chain_type,caip2,evm_chain_id,deployment,chain_name,description,da_layer,symbol,...,l2beat_id,raas,stack,website,twitter,block_explorer,block_explorers,rhino_listed,rhino_naming,origin_key
0,Ethereum,ethereum,L1,eip155-1,1.0,PROD,Ethereum,"Ethereum, proposed by Vitalik Buterin in 2013 ...",None,ETH,...,ethereum,Self-hosted,"{'label': 'Custom', 'url': None}",https://ethereum.org/,https://twitter.com/ethereum,https://etherscan.io/,"{'Etherscan': 'https://etherscan.io/', 'Blocks...",True,ETHEREUM,ethereum
1,All L2s,all-l2s,-,None,NaN,PROD,-,None,None,None,...,None,None,None,None,None,None,None,False,None,all_l2s
2,Arbitrum One,arbitrum,L2,eip155-42161,42161.0,PROD,Arbitrum,Arbitrum One is developed by Offchain Labs. It...,Ethereum (blobs),ARB,...,arbitrum,Self-hosted,"{'label': 'Arbitrum Nitro', 'url': 'https://ar...",https://arbitrum.io/,https://twitter.com/arbitrum,https://arbiscan.io/,"{'Arbiscan': 'https://arbiscan.io/', 'Blocksco...",True,ARBITRUM,arbitrum
3,Base,base,L2,eip155-8453,8453.0,PROD,Base,"Base is an Ethereum Layer 2 offering a secure,...",Ethereum (blobs),None,...,base,Self-hosted,"{'label': 'OP Stack, Superchain', 'url': 'http...",https://base.org/,https://twitter.com/base,https://basescan.org/,"{'BaseScan': 'https://basescan.org/', 'Blocksc...",True,BASE,base
4,Blast,blast,L2,eip155-81457,81457.0,PROD,Blast,Blast is an EVM-compatible Optimistic Rollup w...,Ethereum (blobs),BLAST,...,blast,Self-hosted,"{'label': 'OP Stack', 'url': 'https://docs.opt...",https://blast.io/en,https://twitter.com/Blast_L2,https://blastscan.io/,{'Blast Explorer': 'https://blastscan.io/'},True,BLAST,blast


In [7]:
gtp_fun[gtp_fun['date']>='2024-10-01'].sample(5)

metric_key,index,origin_key,date,costs_blobs_eth,costs_blobs_usd,costs_l1_eth,costs_l1_usd,costs_total_eth,costs_total_usd,daa,...,profit_usd,rent_paid_eth,rent_paid_usd,stables_mcap,stables_mcap_eth,tvl,tvl_eth,txcosts_median_eth,txcosts_median_usd,txcount
3254,4852,ethereum,2024-10-01,NaN,NaN,NaN,NaN,NaN,NaN,431815.0,...,NaN,NaN,NaN,8.059917e+10,3.103142e+07,NaN,NaN,0.000905,2.350125,1266656.0
5984,8481,loopring,2024-10-02,NaN,NaN,0.221426,542.854735,0.221426,542.854735,253.0,...,NaN,0.221426,542.854735,4.371570e+06,1.783127e+03,4.422319e+07,1.803827e+04,NaN,NaN,1891.0
9339,12395,optimism,2024-10-14,1.815347e-10,4.478209e-07,0.340083,838.937412,0.340083,838.937412,93636.0,...,20125.287781,0.340083,838.937412,1.168366e+09,4.736245e+05,6.216726e+09,2.520096e+06,0.000001,0.002800,877568.0
1009,1009,arbitrum,2024-10-06,1.993605e-10,4.815361e-07,0.979683,2366.330445,0.979683,2366.330446,403470.0,...,5665.059346,0.979683,2366.330446,4.515601e+09,1.869501e+06,1.333014e+10,5.518803e+06,0.000002,0.004057,1217061.0
14632,18903,zksync_era,2024-10-17,1.677722e-11,4.381706e-08,0.506231,1322.123638,0.506231,1322.123638,31081.0,...,1802.875497,0.506231,1322.123638,5.765723e+07,2.207651e+04,8.565529e+08,3.279676e+05,0.000006,0.015363,124297.0


### Run L2Beat
[L2Beat](https://l2beat.com/) provides high-level metrics (transactions, assets onchain) and security evaluations for a wide set of L2s and L3s.

In [8]:
print('getting l2beat summary data')
l2b_summary = l2b.get_l2beat_summary()
print('getting l2beat activity data')
l2b_activity = l2b.get_all_projects_data(l2b_summary, 'activity',query_range='max')

print('getting l2beat assets onchain data')
l2b_assets = l2b.get_all_projects_data(l2b_summary, 'tvl',query_range='max')

#filter dates
l2b_activity = l2b_activity[l2b_activity['timestamp']>=start_date]
l2b_assets = l2b_assets[l2b_assets['timestamp']>=start_date]
print('yay')

getting l2beat summary data
https://l2beat.com/api/scaling/summary
getting l2beat activity data
activity API, Chains to run: 113
25 / 113 completed
50 / 113 completed
75 / 113 completed
100 / 113 completed
All projects completed
getting l2beat assets onchain data
tvl API, Chains to run: 113
25 / 113 completed
50 / 113 completed
75 / 113 completed
100 / 113 completed
All projects completed
yay


In [9]:
num_chains = len(l2b_summary[(l2b_summary['isArchived']==False) &(l2b_summary['isUpcoming']==False)])
print(f"L2Beat: Total Amount of Chains: {num_chains}")
l2b_summary.sample(5)

L2Beat: Total Amount of Chains: 113


,id,name,slug,type,category,provider,purposes,isArchived,isUpcoming,isUnderReview,badges,tvl,stage,risks,shortName,hostChain
42,kroma,Kroma,kroma,layer2,Optimistic Rollup,OP Stack,[Universal],False,False,False,"[{'category': 'VM', 'name': 'EVM'}, {'category...","{'breakdown': {'total': 17440899.4, 'ether': 5...",Stage 0,"[{'name': 'Sequencer Failure', 'value': 'Self ...",NaN,NaN
54,astarzkevm,Astar zkEVM,astarzkevm,layer2,Validium,Polygon,[Universal],False,False,False,"[{'category': 'DA', 'name': 'DAC'}, {'category...","{'breakdown': {'total': 3348681.67, 'ether': 1...",NotApplicable,"[{'name': 'Sequencer Failure', 'value': 'No me...",NaN,NaN
12,metis,Metis Andromeda,metis,layer2,Optimium,OVM,[Universal],False,False,False,"[{'category': 'VM', 'name': 'EVM'}, {'category...","{'breakdown': {'total': 260210407.27, 'ether':...",NotApplicable,"[{'name': 'Sequencer Failure', 'value': 'Enque...",Metis,NaN
33,kinto,Kinto,kinto,layer2,Optimistic Rollup,Arbitrum,"[Universal, RWA]",False,False,False,"[{'category': 'RaaS', 'name': 'Caldera'}, {'ca...","{'breakdown': {'total': 34810170.18, 'ether': ...",Stage 0,"[{'name': 'Sequencer Failure', 'value': 'Self ...",NaN,NaN
78,myria,Myria,myria,layer2,Validium,StarkEx,"[NFT, Exchange, Gaming]",False,False,False,"[{'category': 'VM', 'name': 'AppChain'}, {'cat...","{'breakdown': {'total': 133244.12, 'ether': 13...",NotApplicable,"[{'name': 'Sequencer Failure', 'value': 'Force...",NaN,NaN


In [10]:
l2b_activity[l2b_activity['timestamp']>='2024-10-01'].sample(5)

,timestamp,transactions_per_day,slug
27333,2024-11-01,6607,degen
34322,2024-10-16,57041,ham
11953,2024-10-06,31762,zklinknova
20576,2024-10-19,171,karak
37079,2024-10-06,143,clique


In [11]:
l2b_assets[l2b_assets['timestamp']>='2024-10-01'].sample(5)

,timestamp,native,canonical,external,ethPrice,slug
39588,2024-10-31,0.0,37938.91,0.00,2659.2920,wirex
22820,2024-10-14,0.0,18734371.36,0.00,2467.7925,zora
30294,2024-10-11,0.0,30735.29,2105110.89,2383.5962,polynomial
35620,2024-10-06,0.0,43341.75,0.00,2415.4038,aleph-zero
20479,2024-11-05,0.0,235808.21,34614459.90,2403.1309,kinto


### Run Defillama
[Defillama](https://defillama.com/) specializes in TVL (total value locked) data, and also provides high-level metrics for a nearly exhaustive list of chains and applications across ecosystems.

See [API Docs](https://defillama.com/docs/api)

In [12]:
print('getting defillama chain list')
dfl_chains = dfl.get_chain_list()
print('getting defillama chain tvls')
dfl_tvl = dfl.get_all_chains_historical_tvl(dfl_chains)
#filter dates
dfl_tvl = dfl_tvl[dfl_tvl['date']>=start_date]
# filter tvl 
dfl_tvl = dfl_tvl[dfl_tvl['tvl']>0]
print('yay')


getting defillama chain list
getting defillama chain tvls
Defillama API, Chains to run: 313
25 / 313 completed
50 / 313 completed
75 / 313 completed
100 / 313 completed
125 / 313 completed
150 / 313 completed
175 / 313 completed
200 / 313 completed
225 / 313 completed
250 / 313 completed
275 / 313 completed
300 / 313 completed
yay


In [13]:
num_chains = len(dfl_chains)
print(f"Defillama: Total Amount of Chains: {num_chains}")
dfl_chains.sample(5)

Defillama: Total Amount of Chains: 313


,name,chainId
306,Dymension,None
198,Vision,None
245,Concordium,None
261,Elysium,None
39,Conflux,None


In [14]:
dfl_tvl[dfl_tvl['date']>'2024-10-01'].sample(5)

,date,tvl,name,chain_id
208430,2024-10-29,5.129914e+04,Newton,None
162368,2024-10-23,1.002295e+06,Ultron,1231
59650,2024-11-03,6.789380e+05,Evmos,9001
195176,2024-10-21,1.006967e+07,EOS EVM,None
213464,2024-10-06,2.436711e+06,Oasis Sapphire,23294


### Export Data
For this demo, we'll export to local csv files.

In practice, you could also push files to a database

In [15]:
folder_name = 'downloads'

# Check if the folder exists
if not os.path.exists(folder_name):
    # If it doesn't exist, create it
    os.makedirs(folder_name)
    print(f"Folder '{folder_name}' created successfully.")
else:
    print(f"Folder '{folder_name}' already exists.")

Folder 'downloads' already exists.


In [16]:
# Temporary function to store our exporting format more cleanly
def write_df_to_csv(df, file_name):
    df.to_csv(f'{folder_name}/{file_name}.csv', index=False)

#Export Files
print('export growthepie')
write_df_to_csv(gtp_md,'growthepie_metadata')
write_df_to_csv(gtp_fun,'growthepie_fundamentals')

print('export l2beat')
write_df_to_csv(l2b_summary,'l2beat_summary')
write_df_to_csv(l2b_activity,'l2beat_activity')
write_df_to_csv(l2b_assets,'l2beat_assets_onchain')

print('export defillama')
write_df_to_csv(dfl_chains,'defillama_chains')
write_df_to_csv(dfl_tvl,'defillama_tvl')

print('yay')

export growthepie
export l2beat
export defillama
yay
